In [12]:
from PIL import Image
import json
import glob
import pandas as pd
 
# image_name = "image_00026"
json_folder = "yellowjson"
label_folder = "yellowlabels"
split_folder = "yellowlabelsplit"
json_split_folder = "yellowlabelsplitjson"

for name in glob.glob(json_folder + "/*.json"):
#     print(name)
    image_path = label_folder + "/" + name[-16:-5] + ".png"
    json_path = name
    out_path = split_folder + "/" + name[-16:-5]

    #Added a separate out path for saving the json files.
    out_json_path = json_split_folder + '/'

    

    img = Image.open(image_path).convert("RGB")
    width = img.width
    height = img.height
    px = img.load()

    f = open(json_path)
    labels = json.load(f)
    materials = []
    for rows in labels:
        for columns in labels[rows]:
            material = labels[rows][columns]
            if material not in materials:
                materials.append(material)
    materials.remove("Material93")

    #Here's where I create another json file -> for each image, I need a corresponding json that maps to sub image given a label.
    #Therefore, there will be a json file called image_xxxxx_segmentation.json that maps a structure to a split image (e.g. image_xxxxx_x.png)
    labels_json_list = {}

    for index in range(len(materials)):
        structure = materials[index]
        image = Image.new("RGBA", (width, height))
#         print(structure)
        for rows in labels:
            for columns in labels[rows]:
                if labels[rows][columns] == structure:
                    pixel = img.getpixel((int(rows), int(columns)))
                    image.putpixel((int(rows), int(columns)), pixel)
                else:
                    image.putpixel((int(rows), int(columns)), (0, 0, 0, 0))

        image.save(out_path + "_" + str(index) + "_" + ".png")

        #Append to a the dict: label->image to create the json
        labels_json_list[structure] = name[-16:-5] + '_' + str(index) + "_" + ".png"
        
        
        

    #Save the new json file to a folder. 
    #First convert to a DF to make saving to json easier
    splits_json_df = pd.Series(labels_json_list)
    splits_json_df.to_json(out_json_path + name[-16:-5] + '_splits.json')
    #print(out_json_path + name[-16:-5] + '_splits.json')
